# Network of Thrones
### The Weighted Edges of the Storm of Swords Network

A. Beveridge and J. Shan, "Network of Thrones," Math Horizons Magazine , Vol. 23, No. 4 (2016), pp. 18-22.
https://www.macalester.edu/~abeverid/thrones.html

Images courtesy of the Game of Thrones Wiki.

Leverages SAND, py2cytoscape, and igraph to implement the following features:

* An edge's thickness represents its weight.
* The color of a vertex's border indicates its community.
* The size of a vertex corresponds to its PageRank value.
* The size of its label corresponds to its betweenness centrality.
* Degree-sorted circle layout based on group membership, with saved and reloadable manual positions.

## Network Analysis

In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil as su
import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

from IPython.display import Image

import igraph as igraph

from sand.csv import csv_to_dicts
import sand.graph as graph
import sand.cytoscape.positions as scp
import sand.cytoscape.app as app
from sand.cytoscape.themes import ops, colors
import sand.groups as groups

<IPython.core.display.Javascript object>

In [2]:
vertex_data = csv_to_dicts('./data/storm_of_swords_vertices.csv')

In [3]:
edge_data = csv_to_dicts('./data/storm_of_swords_edges.csv')

In [4]:
g = graph.from_vertices_and_edges(
                    vertices=vertex_data, 
                    edges=edge_data, 
                    vertex_name_key='name', 
                    vertex_id_key='name', 
                    edge_foreign_keys=('source_v', 'target_v'),
                    directed=False)
g.summary()

'IGRAPH UNW- 107 352 -- \n+ attr: group (v), imageurl (v), indegree (v), label (v), name (v), outdegree (v), wiki (v), source_v (e), target_v (e), weight (e)'

In [5]:
# Ensure weight doesn't get added as a string
g.es['weight'] = list(map(lambda x: int(x), g.es['weight']))

### Community Detection / Clustering

In [6]:
# calculate dendrogram
dendogram = g.community_edge_betweenness(directed=False, weights='weight')
# convert it into a flat clustering
clusters = dendogram.as_clustering()
# get the membership vector
eb_membership = clusters.membership
len(set(eb_membership))

4

The edge betweenness algorithm finds fewer than the seven communities reported by the authors. We can try a more computationally intensive method, but note this network is close to the size limit of when this algorithm would apply. Heuristic methods like spinglass will get similar results for this type of undirected network in a fraction of the time.

https://en.wikipedia.org/wiki/Modularity_(networks)

In [7]:
om_modules = g.community_optimal_modularity(weights='weight')

In [8]:
om_membership = om_modules.membership

In [9]:
len(set(om_membership))

7

In [10]:
# Represent groups as Strings to avoid a mapping bug in py2cytoscape.
g.vs['group'] = list(map(lambda x: str(x), om_membership))

### Centrality Metrics

In [11]:
g.vs['pagerank'] = g.pagerank(directed=False, weights='weight')

In [12]:
g.vs['betweenness'] = g.betweenness(directed=False, weights='weight')

## Visualize the network in Cytoscape

In [13]:
app.print_version()

{
  "apiVersion": "v1",
  "cytoscapeVersion": "3.4.0"
}


In [14]:
cy = CyRestClient()
cy.session.delete()

In [15]:
network = cy.network.create_from_igraph(g, name='storm of swords', collection='got')

## Customize the style

In [16]:
from sand.cytoscape.themes import colors as c
from sand.cytoscape.themes import label_positions as p

settings = {
    # node style
    'NODE_TRANSPARENCY': 255,
    'NODE_SIZE': 25,
    'NODE_BORDER_WIDTH': 4,
    'NODE_BORDER_PAINT': '#FFCC66',
    'NODE_FILL_COLOR': c.DARK_GREEN,
    'NODE_SELECTED_PAINT': c.BRIGHT_YELLOW,
    'NODE_SHAPE': 'RECTANGLE',

    # node label style
    'NODE_LABEL_COLOR': c.BRIGHT_GRAY,
    'NODE_LABEL_FONT_SIZE': 16,
    'NODE_LABEL_POSITION': p.LOWER_RIGHT,

    # edge style
    'EDGE_TRANSPARENCY': 255,
    'EDGE_WIDTH': 2.5,
    'EDGE_LINE_TYPE': 'SOLID',
    'EDGE_STROKE_SELECTED_PAINT': c.BRIGHT_YELLOW,
    'EDGE_STROKE_UNSELECTED_PAINT': c.BRIGHT_GRAY,
    'EDGE_TARGET_ARROW_UNSELECTED_PAINT': c.BRIGHT_GRAY,

    # network style
    'NETWORK_BACKGROUND_PAINT': c.DARK_GRAY
}

style = cy.style.create('GoT')
style.update_defaults(settings)

In [17]:
# Map the label property in the igraph data to Cytoscape's NODE_LABEL visual property
style.create_passthrough_mapping(column='label', vp='NODE_LABEL', col_type='String')

In [18]:
# Add icons to vertices
style.create_passthrough_mapping(column='imageurl', vp='NODE_CUSTOMGRAPHICS_1', col_type='String')

In [19]:
# scale edges by weight
weight_to_width = su.create_slope(min=min(g.es['weight']), max=max(g.es['weight']), values=(1, 10))
style.create_continuous_mapping(column='weight', vp='EDGE_WIDTH', col_type='Double', points=weight_to_width)

In [20]:
# scale vertex size according to PageRank value.
pr_to_size = su.create_slope(min=min(g.vs['pagerank']), max=max(g.vs['pagerank']), values=(25, 125))
style.create_continuous_mapping(column='pagerank', vp='NODE_SIZE', col_type='Double', points=pr_to_size)

In [21]:
# The size of its label corresponds to its betweenness centrality.
bc_to_size = su.create_slope(min=min(g.vs['betweenness']), max=max(g.vs['betweenness']), values=(14, 32))
style.create_continuous_mapping(column='betweenness', col_type='Double', vp='NODE_LABEL_FONT_SIZE', points=bc_to_size)

In [22]:
# Give each group a unique border color using the number of groups determined above.
border_colors = {
  '0': colors.BRIGHT_YELLOW,
  '1': colors.BRIGHT_RED,
  '2': colors.BRIGHT_BLUE,
  '3': colors.BRIGHT_GREEN,
  '4': colors.BRIGHT_ORANGE,
  '5': colors.BRIGHT_PURPLE,
  '6': colors.BRIGHT_WHITE
}

style.create_discrete_mapping(column='group', col_type='String', vp='NODE_BORDER_PAINT', mappings=border_colors)

In [23]:
cy.style.apply(style, network)

In [24]:
# Apply layout
cy.layout.apply(name='kamada-kawai', network=network)
cy.layout.fit(network)

In [25]:
# Load previous positions
positions_file = './data/storm_of_swords_positions.csv'
scp.layout_from_positions_csv(network, positions_file, cy)
cy.layout.fit(network)

### Save the positions of the vertices to support later updates

In [26]:
scp.positions_to_csv(network=network, path=positions_file)

### Export the view for use in cytoscape.js based visualizations

In [30]:
import json

net_view = network.get_first_view()
with open('./output/storm_of_swords.json', 'w') as out:
    json.dump(net_view, out, indent=2)

In [31]:
style_for_js = cy.style.get(style.get_name(), data_format='cytoscapejs')
with open('./output/got.style.json', 'w') as out:
    json.dump(style_for_js, out, indent=2)